### Почта для сдачи решений:  
    bdt-mf-ml-nlp-2020-q4@bigdatateam.org

**В теме письма укажите:**
    
    "HW2:CompLing. ФИО"

### Пожалуйста, оцените качество заданий второго модуля [здесь](http://rebrand.ly/mfnlp2020q4_feedback_hw02)
-----------------------------------------------------------------------------------------------------------------

# Dependency Parsing

Мы будем использовать **POS tagging** и **dependency parsing** из популярного пакета [SpaCy](https://spacy.io)

В SpaCy используется подход Universal Dependencies

## Установим модель SpaCy

In [3]:
!pip install spacy

In [6]:
!python -m spacy download en

В SpaCy есть возможность не только размечать части речи и синтаксическиезависимости, но и красиво визуализировать, используя ``displacy``

In [7]:
import spacy
from spacy import displacy
nlp = spacy.load('en')
test_doc = nlp('This is a test.')
displacy.render(test_doc, jupyter = True)

### Spacy сама токенизирует текст, все токены становятся специальными элементами, в которых хранятся сам токен,а также его таги

In [8]:
print([(token, token.pos_) for token in test_doc])

[(This, 'DET'), (is, 'AUX'), (a, 'DET'), (test, 'NOUN'), (., 'PUNCT')]


In [9]:
print(test_doc)

This is a test.


### Для задания мы используем известный корпус литературных произведений - Project Gutenberg. 
Из корпуса мы возьмем довольно длинный роман Джейн Остин "Чувство и чувствительность" (Sense and Sensibility)

In [10]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
sentences = gutenberg.sents('austen-sense.txt')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/antonina.goryacheva/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [11]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

### Вопросы

**1. Сколько предложений в этом романе? Сколько уникальных токенов?**

In [13]:
# предложений
len(sentences)

4999

In [14]:
# уникальных тоокенов
len(set([word.lower() for text in sentences for word in text]))

6398

**2. Найдите 5 наиболее частотных глагольных словоформ? Найдите 5 наиболее частотных лемм (начальных форм) глаголов? (глагол-verb)**

In [33]:
from collections import Counter

In [25]:
full_tokens_list = []
for s in sentences:
    doc = nlp(' '.join(s))
    tokens = [(token, token.pos_) for token in doc]
    full_tokens_list.append(tokens)

In [28]:
flatten_tokens = [pair for text in full_tokens_list for pair in text]

In [35]:
# 5 наиболее частотных глагольных словоформ
Counter([x[0] for x in flatten_tokens if x[1] == 'VERB']).most_common(5)

[(settled, 1), (lived, 1), (engage, 1), (surrounding, 1), (lived, 1)]

In [52]:
# 5 наиболее частотных лемм глаголов
Counter([x[0].lemma_ for x in flatten_tokens if x[1] == 'VERB']).most_common(5)

[('say', 609), ('could', 568), ('would', 507), ('may', 384), ('see', 384)]

## Грамматика зависимостей (Dependency parsing) and PP attachment

Теперь посмотрим как глаголы и предложные фразы (prepositional phrase или PP) могут быть связаны в предложении.
- Вершина *предложной фразы* - предлог. Например, "**in** the house", "**on** the table", "**with** my friend"


### Вопросы

  3. В чем отличие в зависимости предложных фраз между предложениями группы (A) и предложениями группы (B)? 
  Посмотрите на схему зависимостей, которую можно сделать с помощью SpaCy.
  - Подсказка: какие "родительские" вершины у предложных групп?
  
(A)
  * I eat an apple in my room.
  * We listen to music at the theater.
  * John visited Brazil with his friend.
  
(B)
  * I see a fly in my soup.
  * She knows the man at the store.
  * I photographed a man with a bowtie.
  

In [ ]:
# Ответ: в группе А предложные фразы зависят от сказуемого, а в группе В - от дополнения.

In [56]:
A = ['I eat an apple in my room.', 'We listen to music at the theater.', 'John visited Brazil with his friend.']
for text in A:
    print(text)
    displacy.render(nlp(text), jupyter = True)

I eat an apple in my room.


We listen to music at the theater.


John visited Brazil with his friend.


In [57]:
A = ['I see a fly in my soup.', 'She knows the man at the store.', 'I photographed a man with a bowtie.']
for text in A:
    print(text)
    displacy.render(nlp(text), jupyter = True)

I see a fly in my soup.


She knows the man at the store.


I photographed a man with a bowtie.


### Бывают случаи, когда не очевидно, к какому элементу предложения нужно присоединить предложную фразу
- Вспомните пример из лекции "I shot an elephant in my pyjamas"

На примере нашего корпуса попробуем понять, какую "родительскую" вершину для предложных фраз предпочитает SpaCy


**Вопросы:**

  4. Выберите все случаи, когда предложная фраза присоединяется к глаголу (verb), как в группе предложений (A) выше.
      - Создайте список для всех пар (tupples) вида (лемматизированный глагол, лемматизированный предлог) = (verb lemma, preposition lemma).
      - Посчитайте сколько таких случаев.
      - Выпишите 5 наиболее частотный пар (verb lemma, preposition lemma)
      - Подсказка: В spaCy можно получить зависимый токен  с помощью *token*.children, а тип зависимости с помощью *child.dep_*. 

In [87]:
a_pairs = []
for s in sentences:
    doc = nlp(' '.join(s))
    for token in doc:
        if token.pos_ == 'VERB':
            for child in token.children:
                if child.dep_ == 'prep':
                    a_pairs += [(token.lemma_, child.lemma_)]

In [88]:
# кол-во пар (verb lemma, preposition lemma)
len(a_pairs)

6030

In [89]:
# 5 наиболее частотных пар (verb lemma, preposition lemma)
Counter(a_pairs).most_common(5)

[(('think', 'of'), 63),
 (('talk', 'of'), 55),
 (('come', 'to'), 51),
 (('go', 'to'), 45),
 (('see', 'in'), 41)]

5. Сделайте тоже самое для случаев, когда предложная фраза присоединяется к дополнению глагола (verb's object), как в группе предложений (B). 
      - Создайте список для всех пар (tupples) вида (лемматизированный дополнение, лемматизированный предлог) = (verb's object lemma, preposition lemma).
      - Посчитайте сколько таких случаев.
      - Выпишите 5 наиболее частотный пар (verb's object, preposition lemma)?
  ?

In [99]:
b_pairs = []
for s in sentences:
    doc = nlp(' '.join(s))
    for token in doc:
        if token.pos_ == 'VERB':
            for child in token.children:
                if child.dep_ == 'dobj':
                    for child2 in child.children:
                        if child2.dep_ == 'prep':
                            b_pairs += [(child.lemma_, child2.lemma_)]

In [100]:
# кол-во пар (verb's object lemma, preposition lemma)
len(b_pairs)

1019

In [101]:
# 5 наиболее частотных пар (verb's object lemma, preposition lemma)
Counter(b_pairs).most_common(5)

[(('nothing', 'of'), 18),
 (('account', 'of'), 10),
 (('necessity', 'of'), 9),
 (('opinion', 'of'), 8),
 (('deal', 'of'), 8)]

6. Постройте систему зависимостей для предложения "I shot an elephant in my pyjamas"
      - К чему присоединена предложная группа?
      - Кто был в пижаме с точки зрения SpaCy?

In [ ]:
# Ответ: предложная группа присоединена к глаголу, в пижаме был стрелок

In [93]:
doc = nlp("I shot an elephant in my pyjamas")
displacy.render(doc, jupyter = True)

**Бонус:** Выберите несколько случайных предложений из корпуса. Постройте для них систему зависимостей. 
- Похоже на правду?

In [94]:
import random

In [96]:
idx = random.choices(range(len(sentences)), k=5)

In [98]:
for i in idx:
    text = ' '.join(sentences[i])
    displacy.render(nlp(text), jupyter = True)

In [ ]:
# Ответ: в целом, на правду похоже.

Решение присылйте на почту  bdt-mf-ml-nlp-2020-q4@bigdatateam.org  

В теме письма укажите: ``HW2:CompLing. ФИО``

Пожалуйста, оставьте обратную связь о задании [по ссылке](http://rebrand.ly/mfnlp2020q4_feedback_hw02). Она (при желании) анонимна ;)